# Using `surprise`

See the documentation [here](https://surprise.readthedocs.io/en/stable/getting_started.html)!

In [3]:

python setup.py clean


SyntaxError: invalid syntax (1935145249.py, line 1)

In [2]:
pip install surprise


  Obtaining dependency information for surprise from https://files.pythonhosted.org/packages/61/de/e5cba8682201fcf9c3719a6fdda95693468ed061945493dea2dd37c5618b/surprise-0.1-py2.py3-none-any.whl.metadata
  Using cached surprise-0.1-py2.py3-none-any.whl.metadata (327 bytes)
  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py) ... done
Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [75 lines of output]
      /private/var/folders/b3/sw4s_d3s4d5frrqwl56qmt7r0000gn/T/pip-install-6_mmof4p/scikit-surprise_dbf198be3e1e46bf8942ec742b11d6bb/setup.py:65: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are u

Failed to build scikit-surprise
ERROR: Could not build wheels for scikit-surprise, which is required to install pyproject.toml-based projects
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pip
pip install --upgrade setuptools


In [1]:
import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

ModuleNotFoundError: No module named 'surprise'

## Agenda

SWBAT:

- use the `surprise` package to build recommendation engines.

In [ ]:
data = surprise.Dataset.load_builtin('ml-100k')

Now that we've downloaded the data, we can find it in a hidden directory:

In [ ]:
df = pd.read_csv('~/.surprise_data/ml-100k/ml-100k/u.data',
            sep='\t', header=None)
df = df.rename(columns={0: 'user', 1: 'item', 2: 'rating', 3: 'timestamp'})
df

## Data Exploration

In [ ]:
df['user'].nunique()

In [ ]:
df['item'].nunique()

In [ ]:
stats = df[['rating', 'timestamp']].describe()
stats

In [ ]:
print(dt.datetime.fromtimestamp(stats.loc['min', 'timestamp']))
print(dt.datetime.fromtimestamp(stats.loc['max', 'timestamp']))

In [ ]:
read = surprise.Reader('ml-100k')

In [ ]:
read.rating_scale

## Modeling

In [ ]:
train, test = surprise.model_selection.train_test_split(data, random_state=42)

In [ ]:
model = KNNBasic().fit(train)

$\hat{r}_{ui} = \frac{
    \sum\limits_{v \in N^k_i(u)} \text{sim}(u, v) \cdot r_{vi}}
    {\sum\limits_{v \in N^k_i(u)} \text{sim}(u, v)}$
    OR
$\hat{r}_{ui} = \frac{
    \sum\limits_{j \in N^k_u(i)} \text{sim}(i, j) \cdot r_{uj}}
    {\sum\limits_{j \in N^k_u(i)} \text{sim}(i, j)}$

In [ ]:
model2 = SVD().fit(train)

$\sum_{r_{ui} \in R_{train}} \left(r_{ui} - \hat{r}_{ui} \right)^2 +
    \lambda\left(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2\right)$

In [ ]:
model3 = NMF().fit(train)

$\hat{r}_{ui} = q_i^Tp_u$

In [ ]:
#most similiar to movie id 51 using from KNNBasic
model.get_neighbors(iid=51, k=1)

In [ ]:
conds = [df['item'] == 51, df['item'] == 65]
choices = 2*[True]

df.loc[np.select(conds, choices, default=False)].sort_values('user')

## Evaluation

In [ ]:
# get predictions
model.test(test)

In [ ]:
#KNNBasic
surprise.accuracy.mae(model.test(test))

In [ ]:
#KNNBasic
surprise.accuracy.rmse(model.test(test))

In [ ]:
#SVD
surprise.accuracy.mae(model2.test(test))

In [ ]:
#SVD
surprise.accuracy.rmse(model2.test(test))

In [ ]:
#NMF
surprise.accuracy.mae(model3.test(test))

In [ ]:
#NMF
surprise.accuracy.rmse(model3.test(test))